In [1]:
#importing all the necessary libraries
from collections import defaultdict
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
from collections import Counter
from google.colab import files
import tarfile
import regex as re
import math
import zipfile
from os import listdir
from os.path import isfile, join
import numpy as np
from itertools import islice
# uploaded = files.upload() #importing tar file

Saving ustawy.tar.gz to ustawy.tar.gz
Saving Wynik.zip to Wynik.zip


In [3]:
# open file
tar = tarfile.open('ustawy.tar.gz')
  
# extracting file
tar.extractall('/content/acts')
bills = [tarinfo.name for tarinfo in tar]  
print(bills[0])
tar.close()

1993_599.txt


Let us define helpful functions

In [4]:
def clean_text(line):
  line = re.sub(r"\s\s+", " ", line)
  line = line.lower()
  return line


def take(n, iterable):
  "Return first n items of the iterable as a list"
  return list(islice(iterable, n))

# Task 1

Use SpaCy tokenizer API to tokenize the text from the law corpus.

In [5]:
nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)
tokenized_bills = defaultdict()

for file_name in bills:
  with open(f'./acts/{file_name}') as file:
    content = file.read()
    cleaned_text = clean_text(content)
    tokens = tokenizer(cleaned_text)
    tokenized_bills[file_name] = [token for token in tokens if not str(token).startswith('_') and str(token)!= ' ']

# Task 2

Compute bigram counts of downcased tokens

In [6]:
bigrams = defaultdict()

for tokens in tokenized_bills.values():
  for idx in range(len(tokens)):
    if idx != len(tokens)-1:
      bigram = str(tokens[idx]) + ' ' + str(tokens[idx+1])

      if bigram in bigrams:
        bigrams[bigram] += 1
      else:
        bigrams[bigram] = 1

In [7]:
take(20,bigrams.items())

[('dz.u. z', 1219),
 ('z 1993', 528),
 ('1993 r.', 771),
 ('r. nr', 17484),
 ('nr 129,', 132),
 ('129, poz.', 130),
 ('poz. 599', 1),
 ('599 ustawa', 1),
 ('ustawa z', 974),
 ('z dnia', 9169),
 ('dnia 9', 210),
 ('9 grudnia', 6),
 ('grudnia 1993', 108),
 ('r. o', 6944),
 ('o zmianie', 1203),
 ('zmianie ustawy', 837),
 ('ustawy o', 1426),
 ('o podatku', 513),
 ('podatku od', 379),
 ('od towarów', 267)]

# Task 3

Discard bigrams containing characters other than letters. Make sure that you discard the invalid entries after computing the bigram counts.

In [8]:
bigrams_updated = defaultdict()

for bigram in bigrams.keys():
  line = re.findall(r"[^\w]|[0-9]", bigram)
  if len(line) == 1: bigrams_updated[bigram] = bigrams[bigram]

In [9]:
take(20,bigrams_updated.items())

[('ustawa z', 974),
 ('z dnia', 9169),
 ('o zmianie', 1203),
 ('zmianie ustawy', 837),
 ('ustawy o', 1426),
 ('o podatku', 513),
 ('podatku od', 379),
 ('od towarów', 267),
 ('towarów i', 525),
 ('i usług', 373),
 ('usług oraz', 122),
 ('oraz o', 926),
 ('podatku akcyzowym', 47),
 ('w ustawie', 4348),
 ('ustawie z', 3648),
 ('i nr', 8146),
 ('wprowadza się', 1766),
 ('się następujące', 1666),
 ('dotychczasowa treść', 37),
 ('treść otrzymuje', 37)]

# Task 4

Use pointwise mutual information to compute the measure for all pairs of words.

In [10]:
#let us define global frequency list of the tokens
frequency_list = defaultdict()

for file_name in tokenized_bills.keys():
  tokens = tokenized_bills[file_name]
  x = [str(i) for i in tokens]
  ctr = Counter(x)
  frequency_list[file_name] = ctr

In [11]:
global_frequency_list = defaultdict()

for file_name in frequency_list.keys():
  tokens = frequency_list[file_name].keys()
  for token in tokens:
    line = re.findall(r"[^\w]|[0-9]", token)
    if len(line) == 0:
      if token in global_frequency_list.keys():
        global_frequency_list[token] += frequency_list[file_name][token]
      else:
        global_frequency_list[token] = frequency_list[file_name][token]
    else:
      pass

In [12]:
no_bigrams = np.sum(list(bigrams_updated.values()))
no_words = np.sum(list(global_frequency_list.values()))

def pmi(bigram, unigram_freq, bigram_freq):
  a,b = bigram.split()
  prob_word1 = unigram_freq[a] / no_words
  prob_word2 = unigram_freq[b] / no_words
  prob_word1_word2 = bigram_freq[bigram] / no_bigrams
  
  return math.log(prob_word1_word2/float(prob_word1*prob_word2),2) 

In [13]:
res = defaultdict()

for bigram in list(bigrams_updated.keys()):
  pmi_bigram = pmi(str(bigram), global_frequency_list, bigrams_updated)
  res[bigram] = pmi_bigram

In [15]:
take(20,res.items())

[('ustawa z', 3.834365041840434),
 ('z dnia', 4.596995990407499),
 ('o zmianie', 5.600710015489752),
 ('zmianie ustawy', 7.82351712593361),
 ('ustawy o', 3.138739983194792),
 ('o podatku', 4.071175082113261),
 ('podatku od', 5.587362531398886),
 ('od towarów', 5.31844462073743),
 ('towarów i', 3.871042282977599),
 ('i usług', 3.7121149588945435),
 ('usług oraz', 3.5278363500971834),
 ('oraz o', 0.7277901569323676),
 ('podatku akcyzowym', 10.528964480031824),
 ('w ustawie', 4.124997379615306),
 ('ustawie z', 5.163173979735109),
 ('i nr', 2.953880679201674),
 ('wprowadza się', 6.257564530439567),
 ('się następujące', 5.987974214893733),
 ('dotychczasowa treść', 12.568094144467164),
 ('treść otrzymuje', 4.920562296549042)]

# Task 5

Sort the word pairs according to that measure in the descending order and determine top 10 entries.

In [16]:
pmi_results = {val[0] : val[1] for val in sorted(res.items(), key = lambda x: (-x[1], x[0]))}

pmi_results_10 = take(10, pmi_results.items())
pmi_results_10

[('admi nistracji', 21.859046996292818),
 ('adminis tracji', 21.859046996292818),
 ('administracjami drogowymi', 21.859046996292818),
 ('aegroti suprema', 21.859046996292818),
 ('aerodynamicznej szorstkości', 21.859046996292818),
 ('agregatach pralniczych', 21.859046996292818),
 ('agricoltura biologica', 21.859046996292818),
 ('agriculture biologique', 21.859046996292818),
 ('agrotechniki nienaruszającej', 21.859046996292818),
 ('ają cyc', 21.859046996292818)]

# Task 6
Filter bigrams with number of occurrences lower than 5. Determine top 10 entries for the remaining dataset (>=5 occurrences).


In [17]:
#filtering
bigrams_5_plus_occurences = defaultdict()

for bigram, count in bigrams_updated.items():
  if count >= 5: bigrams_5_plus_occurences[bigram] = bigrams_updated[bigram]

In [18]:
#counting pmi
result_bigrams_5_plus = defaultdict()

for bigram in list(bigrams_5_plus_occurences.keys()):
  pmi_bigram = pmi(str(bigram), global_frequency_list, bigrams_5_plus_occurences)
  result_bigrams_5_plus[bigram] = pmi_bigram


In [20]:
pmi_results_5_plus_occurences = {val[0] : val[1] for val in sorted(result_bigrams_5_plus.items(), key = lambda x: (-x[1], x[0]))}

pmi_results_5_plus_occurences_10 = take(10, pmi_results_5_plus_occurences.items())
pmi_results_5_plus_occurences_10

[('grzegorz schetyna', 19.537118901405453),
 ('młyny kulowe', 19.537118901405453),
 ('najnowszych zdobyczy', 19.537118901405453),
 ('ręcznego miotacza', 19.537118901405453),
 ('zaszkodzić wynikom', 19.537118901405453),
 ('świeckie przygotowujące', 19.537118901405453),
 ('adama mickiewicza', 19.274084495571664),
 ('chromu sześciowartościowego', 19.274084495571664),
 ('mleczka makowego', 19.274084495571664),
 ('przeponowe rurowe', 19.274084495571664)]

# Task 7

Use KRNNT or Clarin-PL API(https://ws.clarin-pl.eu/tager.shtml) to tag and lemmatize the corpus.


The corpus was uploaded to Clarin-Pl, tagged and lemmatized.

In [21]:
with zipfile.ZipFile('Wynik.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/lemmatized_acts')


In [ ]:
lemmatized_bills = [f for f in listdir('/content/lemmatized_acts')]


In [39]:
lemmatized_tokens = defaultdict()

for bill in lemmatized_bills:
  with open(f'/content/lemmatized_acts/{bill}') as file:
    f = file.read()
    pattern = re.findall(r'\<tok\>\n\s+.*\n.*',f)
    pattern = " ".join(pattern)
    lemmatized_word = re.findall(r'\<base\>(\S+)\<\/base\>\<ctag\>(\w+)', pattern)
    lemmatized_word_cleaned = [word[0] +':' + word[1] for idx, word in enumerate(lemmatized_word) if 
                               len(re.findall(r"[^\w]|[0-9]|_", word[0]))==0 and
                               len(re.findall(r"[^\w]|[0-9]|_", word[1]))==0]
    lemmatized_tokens[bill] = lemmatized_word_cleaned

In [ ]:
lemmatized_tokens['1993_599.txt']

#Task 8

Using the tagged corpus compute bigram statistic for the tokens containing: a. lemmatized, downcased word b. morphosyntactic category of the word (subst, fin, adj, etc.)


In [41]:
bigrams_lemmatized = defaultdict()

for bill in lemmatized_bills:
  for idx in range(len(lemmatized_tokens[bill])):
    if idx != len(lemmatized_tokens[bill])-1:
      bigram = str(lemmatized_tokens[bill][idx]) + ' ' + str(lemmatized_tokens[bill][idx+1])

      if bigram.lower() in bigrams_lemmatized:
        bigrams_lemmatized[bigram.lower()] += 1
      else:
        bigrams_lemmatized[bigram.lower()] = 1

In [42]:
take(10,bigrams_lemmatized.items())

[('u:prep z:prep', 4528),
 ('z:prep r:ign', 19345),
 ('r:ign nr:subst', 17892),
 ('nr:subst poz:ign', 43361),
 ('poz:ign ustawa:subst', 872),
 ('ustawa:subst z:prep', 8777),
 ('z:prep dzień:subst', 11360),
 ('dzień:subst czerwiec:subst', 1401),
 ('czerwiec:subst r:ign', 1349),
 ('r:ign o:prep', 7103)]

# Task 10

Compute the same statistics as for the non-lemmatized words (i.e. PMI) and print top-10 entries with at least 5 occurrences.

In [43]:
#let us define global frequency list of the tokens
frequency_list_lemmatized = defaultdict()

for file_name in lemmatized_bills:
  tokens = lemmatized_tokens[file_name]
  ctr = Counter(tokens)
  frequency_list_lemmatized[file_name] = ctr

In [ ]:
frequency_list_lemmatized['1993_599.txt']

In [45]:
global_frequency_list_lemmatized = defaultdict()

for file_name in lemmatized_bills:
  tokens = frequency_list_lemmatized[file_name].keys()
  for token in tokens:
    if token.lower() in global_frequency_list_lemmatized.keys():
      global_frequency_list_lemmatized[token.lower()] += frequency_list_lemmatized[file_name][token]
    else:
      global_frequency_list_lemmatized[token.lower()] = frequency_list_lemmatized[file_name][token]

In [46]:
no_bigrams_lemmatized = np.sum(list(bigrams_lemmatized.values()))
no_words_lemmatized = np.sum(list(global_frequency_list_lemmatized.values()))

def pmi_lemmatized(bigram, unigram_freq, bigram_freq):
  a,b = bigram.split()
  prob_word1 = unigram_freq[a] / no_words_lemmatized
  prob_word2 = unigram_freq[b] / no_words_lemmatized
  prob_word1_word2 = bigram_freq[bigram] / no_bigrams_lemmatized
  
  return math.log(prob_word1_word2/float(prob_word1*prob_word2),2) 

In [47]:
res_lemmatized = defaultdict()

for bigram in list(bigrams_lemmatized.keys()):
  if bigrams_lemmatized[bigram] >= 5:
    pmi_bigram = pmi_lemmatized(str(bigram), global_frequency_list_lemmatized, bigrams_lemmatized)
    res_lemmatized[bigram] = pmi_bigram

In [ ]:
res_lemmatized

In [48]:
pmi_results_lemmatized = {val[0] : val[1] for val in sorted(res_lemmatized.items(), key = lambda x: (-x[1], x[0]))}

pmi_results_lemmatized_10 = take(10, pmi_results_lemmatized.items())
pmi_results_lemmatized_10

[('błotniarka:subst cedzidło:subst', 19.439453668687708),
 ('denitracyjne:ign odbieralnice:ign', 19.439453668687708),
 ('grzegorz:subst schetyna:ign', 19.439453668687708),
 ('metalizacja:subst natryskowy:adj', 19.439453668687708),
 ('młynek:subst młotkowy:adj', 19.439453668687708),
 ('niklowo:adv kadmowy:adj', 19.439453668687708),
 ('przewoźny:adj dźwignik:subst', 19.439453668687708),
 ('teryto:ign rialnego:ign', 19.439453668687708),
 ('adam:subst mickiewicz:subst', 19.176419262853916),
 ('chrom:subst sześciowartościowy:adj', 19.176419262853916)]

# Task 11

Compute trigram counts for both corpora and perform the same filtering.


Trigrams for Spacy tokens

In [49]:
trigrams = defaultdict()

for tokens in tokenized_bills.values():
  for idx in range(len(tokens)):
    if idx != len(tokens)-1 and idx != len(tokens)-2:
      trigram = str(tokens[idx]) + ' ' + str(tokens[idx+1]) + ' ' + str(tokens[idx+2])

      if trigram in trigrams:
        trigrams[trigram] += 1
      else:
        trigrams[trigram] = 1

In [50]:
trigrams_updated = defaultdict()

for trigram in trigrams.keys():
  line = re.findall(r"[^\w]|[0-9]", trigram)
  if len(line) == 2: trigrams_updated[trigram] = trigrams[trigram]

In [52]:
take(20,trigrams_updated.items())

[('ustawa z dnia', 971),
 ('o zmianie ustawy', 763),
 ('zmianie ustawy o', 629),
 ('ustawy o podatku', 131),
 ('o podatku od', 114),
 ('podatku od towarów', 222),
 ('od towarów i', 252),
 ('towarów i usług', 302),
 ('i usług oraz', 98),
 ('usług oraz o', 56),
 ('oraz o podatku', 48),
 ('o podatku akcyzowym', 39),
 ('w ustawie z', 3392),
 ('ustawie z dnia', 3558),
 ('wprowadza się następujące', 1537),
 ('dotychczasowa treść otrzymuje', 37),
 ('treść otrzymuje oznaczenie', 37),
 ('rada ministrów może', 78),
 ('z wyjątkiem wyrobów', 18),
 ('wyjątkiem wyrobów akcyzowych', 14)]

Trigrams for lemmatized bills


In [53]:
trigrams_lemmatized = defaultdict()

for bill in lemmatized_bills:
  for idx in range(len(lemmatized_tokens[bill])):
    if idx != len(lemmatized_tokens[bill])-1 and idx != len(lemmatized_tokens[bill])-2:
      trigram = str(lemmatized_tokens[bill][idx]) + ' ' + str(lemmatized_tokens[bill][idx+1]) + ' ' + str(lemmatized_tokens[bill][idx+2])

      if trigram.lower() in trigrams_lemmatized:
        trigrams_lemmatized[trigram.lower()] += 1
      else:
        trigrams_lemmatized[trigram.lower()] = 1

In [ ]:
trigrams_lemmatized

# Task 12

Use PMI (with 5 occurrence threshold) to compute top 10 results for the trigrams. Devise a method for computing the values, based on the results for bigrams.

$$PMI_T(x,y,z) := log_2 \frac{P(x,y,z)}{P(x)\cdot P(y) \cdot P(z)} = PMI_B(x,y) + log_2 \frac{P(x,y,z)}{P(z)\cdot P(x,y)}$$

In [55]:
no_trigrams = np.sum(list(trigrams_updated.values()))

def pmi_trigrams(trigram, unigram_freq, bigram_freq, trigram_freq):
  a,b,c = trigram.split()
  bigram = a + ' ' + b
  print()
  pmi_ab = pmi_results[bigram]

  prob_word3 = unigram_freq[c] / no_words
  prob_word1_word2 = bigram_freq[bigram] / no_bigrams
  prob_word1_word2_word3 = trigram_freq[trigram] / no_trigrams

  
  return math.log(prob_word1_word2_word3/float(prob_word3*prob_word1_word2),2) + pmi_ab

In [ ]:
res_trigrams = defaultdict()

for trigram in list(trigrams_updated.keys()):
  if trigrams_updated[trigram] >= 5:
    pmi_trigram = pmi_trigrams(str(trigram), global_frequency_list, bigrams_updated, trigrams_updated)
    res_trigrams[trigram] = pmi_trigram

In [57]:
pmi_results_trigrams = {val[0] : val[1] for val in sorted(res_trigrams.items(), key = lambda x: (-x[1], x[0]))}

pmi_results_trigrams_10 = take(10, pmi_results_trigrams.items())
pmi_results_trigrams_10

[('piłce nożnej uefa', 36.12991870523395),
 ('profilem zaufanym epuap', 36.10343478373326),
 ('religijne uroczystości pogrzebowe', 36.04488260137815),
 ('finałowego turnieju mistrzostw', 36.01841039001695),
 ('potwierdzonym profilem zaufanym', 35.64532856256253),
 ('turnieju mistrzostw europy', 35.639898766763224),
 ('przewozach szybko psujących', 35.281921798679),
 ('stosującym szkodliwą konkurencję', 34.8928795079331),
 ('grożącą jemu samemu', 34.770959879401616),
 ('poddanych szybkiemu postępowi', 34.6798857845989)]

In [58]:
no_trigrams_lemmatized = np.sum(list(trigrams_lemmatized.values()))

def pmi_trigrams_lemmatized(trigram, unigram_freq, bigram_freq, trigram_freq):
  a,b,c = trigram.split()
  bigram = a + ' ' + b
  pmi_ab = pmi_results_lemmatized[bigram]

  prob_word3 = unigram_freq[c] / no_words_lemmatized
  prob_word1_word2 = bigram_freq[bigram] / no_bigrams_lemmatized
  prob_word1_word2_word3 = trigram_freq[trigram] / no_trigrams_lemmatized

  return math.log(prob_word1_word2_word3/float(prob_word3*prob_word1_word2),2) + pmi_ab

In [59]:
res_trigrams_lemmatized = defaultdict()

for trigram in list(trigrams_lemmatized.keys()):
  if trigrams_lemmatized[trigram] >= 5:
    pmi_trigram = pmi_trigrams_lemmatized(str(trigram), global_frequency_list_lemmatized, bigrams_lemmatized, trigrams_lemmatized)
    res_trigrams_lemmatized[trigram] = pmi_trigram

In [60]:
pmi_results_trigrams_lemmatized = {val[0] : val[1] for val in sorted(res_trigrams_lemmatized.items(), key = lambda x: (-x[1], x[0]))}

pmi_results_trigrams_lemmatized_10 = take(10, pmi_results_trigrams_lemmatized.items())
pmi_results_trigrams_lemmatized_10

[('porcelanowy:adj młyn:subst kulowy:adj', 37.39348066910422),
 ('denitracyjne:ign odbieralnice:ign hydrauliczny:adj', 37.20083559116182),
 ('roleta:subst lambrekin:subst okienny:adj', 37.17108824776777),
 ('piran:ign ola:subst þ:ign', 37.08980427877308),
 ('topić:ger żużel:subst wielkopiecowy:adj', 37.06333206741189),
 ('maria:subst curie:subst skłodowska:subst', 36.878907496274465),
 ('þ:ign tetrahydro:ign trimetylo:ign', 36.76787618388572),
 ('wymiennik:subst przeponowy:adj rurowy:adj', 36.52276368604919),
 ('puszek:subst wymiennik:subst przeponowy:adj', 36.45237435815779),
 ('soda:subst chłodnica:subst odmulina:subst', 36.45237435815779)]

# Task 13

Create a table comparing the results for copora without and with tagging and lemmatization (separate table for bigrams and trigrams).


In [61]:
a = [elem[0] for elem in pmi_results_5_plus_occurences_10]
b = [elem[0] for elem in pmi_results_10]
c = [elem[0] for elem in pmi_results_lemmatized_10]

d = [elem[0] for elem in pmi_results_trigrams_10]
e = [elem[0] for elem in pmi_results_trigrams_lemmatized_10]

In [62]:
import pandas as pd
df = pd.DataFrame(list(zip(b,a,c)),
               columns =['bigrams', 'bigrams with more than 5 occurances', 'bigrams lemmatized'])
df

,bigrams,bigrams with more than 5 occurances,bigrams lemmatized
0,admi nistracji,grzegorz schetyna,błotniarka:subst cedzidło:subst
1,adminis tracji,młyny kulowe,denitracyjne:ign odbieralnice:ign
2,administracjami drogowymi,najnowszych zdobyczy,grzegorz:subst schetyna:ign
3,aegroti suprema,ręcznego miotacza,metalizacja:subst natryskowy:adj
4,aerodynamicznej szorstkości,zaszkodzić wynikom,młynek:subst młotkowy:adj
5,agregatach pralniczych,świeckie przygotowujące,niklowo:adv kadmowy:adj
6,agricoltura biologica,adama mickiewicza,przewoźny:adj dźwignik:subst
7,agriculture biologique,chromu sześciowartościowego,teryto:ign rialnego:ign
8,agrotechniki nienaruszającej,mleczka makowego,adam:subst mickiewicz:subst
9,ają cyc,przeponowe rurowe,chrom:subst sześciowartościowy:adj


In [63]:
df_tri = pd.DataFrame(list(zip(d,e)),
               columns =['trigrams with more than 5 occurances', 'trigrams lemmatized'])
df_tri

,trigrams with more than 5 occurances,trigrams lemmatized
0,piłce nożnej uefa,porcelanowy:adj młyn:subst kulowy:adj
1,profilem zaufanym epuap,denitracyjne:ign odbieralnice:ign hydrauliczny...
2,religijne uroczystości pogrzebowe,roleta:subst lambrekin:subst okienny:adj
3,finałowego turnieju mistrzostw,piran:ign ola:subst þ:ign
4,potwierdzonym profilem zaufanym,topić:ger żużel:subst wielkopiecowy:adj
5,turnieju mistrzostw europy,maria:subst curie:subst skłodowska:subst
6,przewozach szybko psujących,þ:ign tetrahydro:ign trimetylo:ign
7,stosującym szkodliwą konkurencję,wymiennik:subst przeponowy:adj rurowy:adj
8,grożącą jemu samemu,puszek:subst wymiennik:subst przeponowy:adj
9,poddanych szybkiemu postępowi,soda:subst chłodnica:subst odmulina:subst


# Task 14

Answer the following questions:


* Why do we have to filter the bigrams, rather than the token sequence?

If we perfomred filtering with token instead of filtering the bigrams, we would have a situation where we take non existing bigrams like for example the sentence "Ala ma kota" would produce a bigram "Ala Kota" which does not exist.



* Which method works better for the bigrams and which for the trigrams?

I will risk stating that the SpAcy tokenizers method works better in both cases.

* What types of expressions are discovered by the methods.

SpAcy tokenizer discovered factual statements/people like "profilem zaufanym epuap", "finałowego turnieju mistrzostw" or "grzegorz schetyna". The Clarin-PL API produced a few terrible outcomes like: piran:ign ola:subst þ:ign but also a few correct statements like "wymiennik:subst przeponowy:adj rurowy:adj" or "grzegorz:subst schetyna:ign". 

* Can you devise a different type of filtering that would yield better results?

I would propose a fitering method which removes stop words and one-lettered words like "p","r" etc. 